# 2.model_structure

In [2]:
import sys
sys.path = ["../../.."] + sys.path # 切换到项目目录下

import scanpy as sc
import scvelo as scv
import velovgi

from ray import tune, air
from ray.air import session

Global seed set to 0


1. 目标函数

In [3]:
from pytorch_lightning import loggers
from torch_geometric import seed_everything

# TODO: 跳整多种参数，简化调参Trail的名字
def train_velovgi(config):
    # 提取参数
    # 随机数种子，确保结果的可复现性
    random_seed = config.get("random_seed", 0)
    # 预处理的参数
    n_bnn_neighbors = config.get("n_bnn_neighbors", 15)
    n_knn_neighbors = config.get("n_knn_neighbors", 15)
    is_ot = config.get("is_ot", True)
    # 模型结构参数
    n_hidden = config.get("n_hidden", 256)
    n_latent = config.get("n_latent", 10)
    n_layers = config.get("n_layers", 1)
    # 训练参数
    num_neighbors = [config.get("num_neighbors", 8)]*n_layers
    max_epochs = config.get("max_epochs", 10) # TODO:这里是最关键的一个参数，小epochs测试之后再提交到服务器上用大epoch
    batch_size = config.get("batch_size", 64)
    max_kl_weight = config.get("max_kl_weight", 0.8)

    name = ""
    for k,v in config.items():
        name += "%s_%s,"%(k, v)
    name = name[:-1]

    # seed_everything(random_seed)
    # # TODO:数据读入，对于不同的数据集这里需要替换
    # adata_filename = "/mnt/h/F_bak/Python进阶/scRNA/Other/velovgi_workstation/notebook/local_pc/erythroid_lineage/data/erythroid_lineage.h5ad" # 数据路径使用绝对路径
    # adata = scv.read(adata_filename)
    # batch_key = "stage" # 批次key
    cluster_key = "celltype" # 细胞类型key
    cluster_edges = [
    ("Blood progenitors 1", "Blood progenitors 2"), 
    ("Blood progenitors 2", "Erythroid1"), 
    ("Erythroid1", "Erythroid2"), 
    ("Erythroid2", "Erythroid3")
    ] # 指定对应数据集已知的细胞类型间的分化信息

    # # TODO:预处理，这里batch_pair_list以后可能需要手动指定
    # batch_list = list(adata.obs[batch_key].cat.categories)
    # batch_pair_list = list(zip(batch_list[:-1], batch_list[1:]))
    # subsample_adata = velovgi.pp.preprocess(adata,
    #                                         n_bnn_neighbors=n_bnn_neighbors,
    #                                         n_knn_neighbors=n_knn_neighbors,
    #                                         batch_key=batch_key,
    #                                         batch_pair_list=batch_pair_list,
    #                                         is_ot=is_ot)
    # TODO: 如果需要调整预处理之后的参数，就不需要重复做预处理了，读取预处理之后的结果即可
    adata = velovgi.tl.read_adata("/mnt/h/F_bak/Python进阶/scRNA/Other/velovgi_workstation/notebook/local_pc/erythroid_lineage/data/adata")
    subsample_adata = scv.read("/mnt/h/F_bak/Python进阶/scRNA/Other/velovgi_workstation/notebook/local_pc/erythroid_lineage/data/subsample_adata.h5ad") # 使用这个AnnData做训练
    seed_everything(random_seed)

    # 模型训练
    logger = loggers.TensorBoardLogger(save_dir="./log", name=name)
    velovgi.tl.VELOVGI.setup_anndata(adata=subsample_adata, spliced_layer="Ms", unspliced_layer="Mu")
    velovgi_model = velovgi.tl.VELOVGI(subsample_adata,
                                       n_hidden=n_hidden,
                                       n_latent=n_latent,
                                       n_layers=n_layers)
    velovgi_model.train(num_neighbors=num_neighbors,
                        max_epochs=max_epochs,
                        batch_size=batch_size,
                        plan_kwargs={"max_kl_weight": max_kl_weight},
                        logger=logger)

    # 模型恢复
    velovgi.tl.add_velovi_outputs_to_adata(subsample_adata, velovgi_model) # 模型输出
    velovgi.pp.moment_recover(adata, subsample_adata) # 恢复

    # 速率计算
    scv.tl.velocity_graph(adata)
    scv.pl.velocity_embedding(adata, color=cluster_key, title=name, save="arrow.png")
    scv.pl.velocity_embedding_stream(adata, color=cluster_key, title=name, legend_loc="right", save="stream.png")

    # 伪时间计算
    scv.tl.velocity_pseudotime(adata)
    scv.pl.velocity_embedding_stream(adata, color="velocity_pseudotime", title=name, colorbar=False, save="pseudotime.png")


    # 保存结果
    subsample_adata.write("subsample_adata.h5ad")
    velovgi.tl.write_adata(adata, "adata")
    velovgi_model.save("model")

    # 计算指标评价
    adata_velo = velovgi.tl.pre_metric(adata)
    exp_metrics = velovgi.tl.summary_metric(adata_velo, cluster_edges, cluster_key)[-1] # 计算指标汇总后的结果

    session.report({"CBDir": exp_metrics["CBDir"], "ICVCoh": exp_metrics["ICVCoh"]})


2. 搜索空间，这里可以添加键值，实现更多层面的网格调参

In [4]:
search_space = {
    "n_hidden" : tune.grid_search([128, 256]),
    # "n_latent" : tune.grid_search([8, 16]),
    # "n_layers" : tune.grid_search([1, 2]),
}

3. 执行调参，等待传入实验名称和搜索空间

In [5]:
from ray.tune.schedulers import ASHAScheduler

name = "model_structure" # TODO:指定此次调参的名字，这里是预处理过程的调参

tuner = tune.Tuner(
    train_velovgi,
    tune_config=tune.TuneConfig(
        metric="CBDir",
        mode="max",
        scheduler=ASHAScheduler()
    ),
    run_config=air.RunConfig(
        local_dir="./results", # Trail内部具体输出结果在这里保存
        name=name # 开启调参的Tensorboard日志
    ),
    param_space=search_space,
)

results = tuner.fit()

2023-06-10 11:03:51,791	INFO worker.py:1625 -- Started a local Ray instance.
2023-06-10 11:03:53,485	INFO tune.py:218 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.
2023-06-10 11:03:53,501	INFO tensorboardx.py:172 -- pip install "ray[tune]" to see TensorBoard files.
2023-06-10 11:03:53,502	WARNING callback.py:142 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


(pid=3905) Global seed set to 0
(pid=3905) /usr/local/conda/envs/velovi-env/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
(pid=3905)   new_rank_zero_deprecation(
(pid=3905) /usr/local/conda/envs/velovi-env/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
(pid=3905)   return new_rank_zero_deprecation(*args, **kwargs)


(train_velovgi pid=3905) load /mnt/h/F_bak/Python进阶/scRNA/Other/velovgi_workstation/notebook/local_pc/erythroid_lineage/data/adata/adata.h5ad
(train_velovgi pid=3905) load /mnt/h/F_bak/Python进阶/scRNA/Other/velovgi_workstation/notebook/local_pc/erythroid_lineage/data/adata/sample_recover.pkl
(train_velovgi pid=3905) 初始训练，初始化runner参数
(train_velovgi pid=3905) choosing neighbor minibatch


(train_velovgi pid=3905) GPU available: False, used: False
(train_velovgi pid=3905) TPU available: False, using: 0 TPU cores
(train_velovgi pid=3905) IPU available: False, using: 0 IPUs
(train_velovgi pid=3905) HPU available: False, using: 0 HPUs
(train_velovgi pid=3905) Missing logger folder: ./log/n_hidden_128


Epoch 2/10:  10%|█         | 1/10 [00:01<00:13,  1.53s/it, loss=1.93e+06, v_num=0]


...

(train_velovgi pid=3969) `Trainer.fit` stopped: `max_epochs=10` reached.


(train_velovgi pid=3905) computing velocity graph (using 1/12 cores)
(train_velovgi pid=3905)   0%|          | 0/500 [00:00<?, ?cells/s]
(train_velovgi pid=3905)     finished (0:00:01) --> added 
(train_velovgi pid=3905)     'velocity_graph', sparse matrix with cosine correlations (adata.uns)
(train_velovgi pid=3905) computing velocity embedding
(train_velovgi pid=3905)     finished (0:00:00) --> added
(train_velovgi pid=3905)     'velocity_umap', embedded velocity vectors (adata.obsm)
(train_velovgi pid=3905) saving figure to file ./figures/scvelo_arrow.png
(train_velovgi pid=3905) Figure(640x480)
(train_velovgi pid=3905) saving figure to file ./figures/scvelo_stream.png
(train_velovgi pid=3905) Figure(640x480)
(train_velovgi pid=3905) saving figure to file ./figures/scvelo_pseudotime.png
(train_velovgi pid=3905) Figure(640x480)
(train_velovgi pid=3905) create adata
(train_velovgi pid=3905) save adata/sample_recover.pkl
(train_velovgi pid=3905) save adata/adata.h5ad
(train_velovgi pid

Trial name,CBDir,ICVCoh,date,done,experiment_tag,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
train_velovgi_6e982_00000,0.45055,0.973809,2023-06-10_11-04-51,True,0_n_hidden=128,DESKTOP-9GVJMSD,1,172.29.205.215,3905,52.2752,52.2752,52.2752,1686366291,1,6e982_00000
train_velovgi_6e982_00001,0.535893,0.971254,2023-06-10_11-04-59,True,1_n_hidden=256,DESKTOP-9GVJMSD,1,172.29.205.215,3969,54.2011,54.2011,54.2011,1686366299,1,6e982_00001


(train_velovgi pid=3969)     finished (0:00:01) --> added 
(train_velovgi pid=3969)     'velocity_graph', sparse matrix with cosine correlations (adata.uns)
(train_velovgi pid=3969) computing velocity embedding
(train_velovgi pid=3969)     finished (0:00:00) --> added
(train_velovgi pid=3969)     'velocity_umap', embedded velocity vectors (adata.obsm)
(train_velovgi pid=3969) saving figure to file ./figures/scvelo_arrow.png
(train_velovgi pid=3969) Figure(640x480)
(train_velovgi pid=3969) saving figure to file ./figures/scvelo_stream.png
(train_velovgi pid=3969) Figure(640x480)
(train_velovgi pid=3969) saving figure to file ./figures/scvelo_pseudotime.png
(train_velovgi pid=3969) Figure(640x480)
(train_velovgi pid=3969) create adata
(train_velovgi pid=3969) save adata/sample_recover.pkl
(train_velovgi pid=3969) save adata/adata.h5ad


2023-06-10 11:04:59,454	INFO tune.py:945 -- Total run time: 65.97 seconds (65.91 seconds for the tuning loop).
